# Prepare

In [1]:
!nvidia-smi

Mon Dec 19 14:42:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    32W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd "/content/drive/MyDrive/CE Project"
!ls

/content/drive/.shortcut-targets-by-id/1-CcB_cv6tez9MQfK-roEW98audOFqMrS/CE Project
 data   softmax.ipynb   svm2.ipynb   SVM.ipynb	'Use Library.ipynb'


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import time
from sklearn.metrics import accuracy_score, f1_score
from keras.datasets import mnist, cifar10
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import OneHotEncoder
import random

In [4]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_everything(22)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define model

In [6]:
def svm_loss(W, X, y, reg):
    d, C = W.shape
    N = X.shape[0]
    loss = 0
    dW = torch.zeros_like(W)
    Z = torch.mm(X, W)
    id0 = range(Z.shape[0])
    X = X.cpu().numpy()
    W = W.cpu().numpy()
    Z = Z.cpu().numpy()
    y = y.cpu().numpy().astype(int)
    correct_class_score = Z[id0, y].reshape(N, 1)
    margins = np.maximum(0, Z - correct_class_score + 1)
    margins[id0, y] = 0
    loss = np.sum(margins)
    loss /= N
    loss += 0.5 * reg * np.sum(W*W)

    F = (margins > 0).astype(int)
    F[np.arange(F.shape[0]), y] = np.sum(-F, axis=1)
    dW = X.T.dot(F)/N + reg*W
    return loss, torch.from_numpy(dW).to(device)

def svm_fit(X, y, Winit, reg, lr=.1, batch_size=32, epochs=100, print_every=5):
    W = torch.clone(Winit)
    loss_hist = []
    n_batches = int(np.ceil(X.shape[0]/batch_size))

    for ep in range(epochs):
        mix_ids = torch.randperm(X.shape[0])
        
        for i in range(n_batches):
            ids = mix_ids[batch_size*i : min(batch_size*(i+1), X.shape[0])]
            X_batch = X[ids]
            y_batch = y[ids]
            loss_i, dW = svm_loss(W, X_batch, y_batch, reg)
            loss_hist.append(loss_i)
            W -= lr*dW

        if ep % print_every == 0 and ep > 0:
            print('Epoch:', ep, 'loss:', loss_hist[-1])
    return W, loss_hist

def svm_predict(W, X):
    Z = X.dot(W)
    return np.argmax(Z, axis=1)

def svm_evaluate(W, X, y):
    y_pred = svm_predict(W, X)
    acc = 100*np.mean(y_pred==y)
    return acc

# CIFAR10

In [7]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, X_test.shape)

170498071/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3)


In [8]:
X_train = np.array(pickle.load(open('/content/drive/MyDrive/CE Project/data/cifar10/train_features.sav', 'rb')))
y_train = np.array(pickle.load(open('/content/drive/MyDrive/CE Project/data/cifar10/y_train.sav', 'rb')))
X_test = np.array(pickle.load(open('/content/drive/MyDrive/CE Project/data/cifar10/test_features.sav', 'rb')))

print(X_train.shape, y_train.shape, X_test.shape)

(10000, 25088) (10000,) (10000, 25088)


In [9]:
max_values = np.max(X_train)
min_values = np.min(X_train)
a, b = 0, 1
X_train = a + (X_train - min_values) * (b-a) / (max_values - min_values)
X_test = a + (X_test - min_values) * (b-a) / (max_values - min_values)

In [10]:
X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)

In [11]:
W_init = torch.randn(X_train.shape[1], np.max(y_test)+1).to(device)

### lambda = 1

In [ ]:
lr = 0.01
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=1, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 8.014969527721405
Epoch: 10 loss: 8.494443356990814
Epoch: 15 loss: 7.576897144317627
Epoch: 20 loss: 8.971588611602783
Epoch: 25 loss: 8.337978661060333
Epoch: 30 loss: 8.562956035137177
Epoch: 35 loss: 8.21283906698227
Epoch: 40 loss: 7.758509993553162
Epoch: 45 loss: 8.746450006961823
Epoch: 50 loss: 8.217262148857117
Epoch: 55 loss: 8.045384049415588
Epoch: 60 loss: 8.029059112071991
Epoch: 65 loss: 8.28987044095993
Epoch: 70 loss: 7.421713948249817
Epoch: 75 loss: 9.05216360092163
Epoch: 80 loss: 8.317854046821594
Epoch: 85 loss: 7.516919553279877
Epoch: 90 loss: 8.122407793998718
Epoch: 95 loss: 8.918424904346466
Train completed in 488.93515729904175s


In [ ]:
svm_evaluate(W.cpu().numpy(), X_test, y_test.reshape(-1))

22.82

### lambda=0.1

In [13]:
lr = 0.01
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=0.1, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 297.39644889831544
Epoch: 10 loss: 18.206084060668946
Epoch: 15 loss: 5.981684017181397
Epoch: 20 loss: 5.811873435974121
Epoch: 25 loss: 4.8997471809387205
Epoch: 30 loss: 4.4718217849731445
Epoch: 35 loss: 4.741220903396607
Epoch: 40 loss: 5.386926317214966
Epoch: 45 loss: 5.304230642318726
Epoch: 50 loss: 5.120079851150512
Epoch: 55 loss: 5.913448715209961
Epoch: 60 loss: 5.422574138641357
Epoch: 65 loss: 5.954001998901367
Epoch: 70 loss: 4.958311748504639
Epoch: 75 loss: 4.390733480453491
Epoch: 80 loss: 5.492269992828369
Epoch: 85 loss: 5.0579413890838625
Epoch: 90 loss: 5.254567003250122
Epoch: 95 loss: 5.526471042633057
Train completed in 470.40967178344727s


In [23]:
svm_evaluate(W.cpu().numpy(), X_test, y_test.reshape(-1))

55.230000000000004

### lambda=0.0001

In [31]:
lr = 0.01
start = time.time()
W, loss_hist = svm_fit(X_train, y_train, W_init, reg=0.0001, lr=lr)
print('Train completed in {}s'.format(time.time() - start))

Epoch: 5 loss: 21.80246057128906
Epoch: 10 loss: 28.870582122802738
Epoch: 15 loss: 22.47023517150879
Epoch: 20 loss: 18.928159350585936
Epoch: 25 loss: 19.944515380859375
Epoch: 30 loss: 20.21176494140625
Epoch: 35 loss: 15.819739279556275
Epoch: 40 loss: 18.320712438964843
Epoch: 45 loss: 15.804937966156006
Epoch: 50 loss: 16.876554835510255
Epoch: 55 loss: 13.241848642730714
Epoch: 60 loss: 16.607401252746584
Epoch: 65 loss: 14.025162215423585
Epoch: 70 loss: 16.13781818084717
Epoch: 75 loss: 17.799059366607665
Epoch: 80 loss: 15.614804837799072
Epoch: 85 loss: 14.093706764221192
Epoch: 90 loss: 13.288495288848878
Epoch: 95 loss: 13.916623931884766
Train completed in 253.32351970672607s


In [32]:
svm_evaluate(W.cpu().numpy(), X_test, y_test.reshape(-1))

45.83